In [7]:
%matplotlib inline
%load_ext autoreload

In [3]:
import os
os.chdir('/home/jaentrouble/adipose_train2')
print(os.getcwd())

/home/jaentrouble/adipose_train2


In [8]:
!git pull
%autoreload

Already up to date.


In [10]:
import numpy as np
import matplotlib.pyplot as plt

In [11]:
with np.load('cell_mask_data.npz') as data:
    X = data['img']
    Y = data['mask']
print('X shape: ', X.shape, X.dtype)
print('Y shape: ', Y.shape, Y.dtype)

X shape:  (1533, 200, 200, 3) uint8
Y shape:  (1533, 200, 200) float32
